In [ ]:
options("scipen"=100, "digits"=4)
if(!require("readr")) install.packages("readr")
if(!require("rpart")) install.packages("rpart")
if(!require("rpart.plot")) install.packages("rpart.plot")
library("readr")
library("rpart")
library("rpart.plot")

<table style="width:98%;">
<colgroup>
<col style="width: 9%" />
<col style="width: 4%" />
<col style="width: 8%" />
<col style="width: 7%" />
<col style="width: 8%" />
<col style="width: 9%" />
<col style="width: 8%" />
<col style="width: 11%" />
<col style="width: 8%" />
<col style="width: 12%" />
<col style="width: 9%" />
</colgroup>
<thead>
<tr class="header">
<th style="text-align: left;">Alternate</th>
<th style="text-align: left;">Bar</th>
<th style="text-align: left;">Fri/Sat</th>
<th style="text-align: left;">Hungry</th>
<th style="text-align: left;">Patrons</th>
<th style="text-align: left;">Price</th>
<th style="text-align: left;">Raining</th>
<th style="text-align: left;">Reservation</th>
<th style="text-align: left;">Type</th>
<th style="text-align: left;">WaitEstimate</th>
<th style="text-align: left;">WillWait</th>
</tr>
</thead>
<tbody>
<tr class="odd">
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">Some</td>
<td style="text-align: left;">Expensive</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">French</td>
<td style="text-align: left;">0-10</td>
<td style="text-align: left;">Yes</td>
</tr>
<tr class="even">
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">Full</td>
<td style="text-align: left;">Cheap</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">Thai</td>
<td style="text-align: left;">30-60</td>
<td style="text-align: left;">No</td>
</tr>
<tr class="odd">
<td style="text-align: left;">No</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">Some</td>
<td style="text-align: left;">Cheap</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">Burger</td>
<td style="text-align: left;">0-10</td>
<td style="text-align: left;">Yes</td>
</tr>
<tr class="even">
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">Full</td>
<td style="text-align: left;">Cheap</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">Thai</td>
<td style="text-align: left;">10-30</td>
<td style="text-align: left;">Yes</td>
</tr>
<tr class="odd">
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">Full</td>
<td style="text-align: left;">Expensive</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">French</td>
<td style="text-align: left;">&gt;60</td>
<td style="text-align: left;">No</td>
</tr>
<tr class="even">
<td style="text-align: left;">No</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">Some</td>
<td style="text-align: left;">Moderate</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">Italian</td>
<td style="text-align: left;">0-10</td>
<td style="text-align: left;">Yes</td>
</tr>
<tr class="odd">
<td style="text-align: left;">No</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">None</td>
<td style="text-align: left;">Cheap</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">Burger</td>
<td style="text-align: left;">0-10</td>
<td style="text-align: left;">No</td>
</tr>
<tr class="even">
<td style="text-align: left;">No</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">Some</td>
<td style="text-align: left;">Moderate</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">Thai</td>
<td style="text-align: left;">0-10</td>
<td style="text-align: left;">Yes</td>
</tr>
<tr class="odd">
<td style="text-align: left;">No</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">Full</td>
<td style="text-align: left;">Cheap</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">Burger</td>
<td style="text-align: left;">&gt;60</td>
<td style="text-align: left;">No</td>
</tr>
<tr class="even">
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">Full</td>
<td style="text-align: left;">Expensive</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">Italian</td>
<td style="text-align: left;">10-30</td>
<td style="text-align: left;">No</td>
</tr>
<tr class="odd">
<td style="text-align: left;">No</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">None</td>
<td style="text-align: left;">Cheap</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">Thai</td>
<td style="text-align: left;">0-10</td>
<td style="text-align: left;">No</td>
</tr>
<tr class="even">
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">Yes</td>
<td style="text-align: left;">Full</td>
<td style="text-align: left;">Cheap</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">No</td>
<td style="text-align: left;">Burger</td>
<td style="text-align: left;">30-60</td>
<td style="text-align: left;">Yes</td>
</tr>
</tbody>
</table>

In [ ]:
url<-"https://docs.google.com/spreadsheets/d/e/2PACX-1vQvQ1hQJoj_3BmOV3uLWcDzwIw3GFhkS84NePBS57lnCmXSN86KU8kn4aQUE9rGsxJPMi32S55JBl9w/pub?gid=0&single=true&output=csv"
traindf<-read.csv(url, stringsAsFactor=TRUE)

Now let’s print out the training set to make sure we read it correctly:

In [ ]:
print(traindf)

Create models for the following individual predictors with maxdepth=1.

In [ ]:
PUT CODE HERE to create a model  WillWait~Price (use maxdepth = 1) 

In [ ]:
PUT CODE HERE to create a model  WillWait~Patrons(use maxdepth = 1) 

Which of the above models is better?

Finally here is a model using all the variables

In [ ]:
control = rpart.control(minsplit=1)
model <- rpart(WillWait~., data=traindf, method="class", control = control)
rpart.plot(model, type=4, extra = 1, digits=-2)

Lets read in the testing data set:

In [ ]:
testurl<-"https://docs.google.com/spreadsheets/d/e/2PACX-1vQvQ1hQJoj_3BmOV3uLWcDzwIw3GFhkS84NePBS57lnCmXSN86KU8kn4aQUE9rGsxJPMi32S55JBl9w/pub?gid=289547774&single=true&output=csv"
testdf<-read.csv(testurl, stringsAsFactors=TRUE)
str(testdf)

Do a prediction for the testing data set using the model above:

In [ ]:
pred <- predict(model, newdata = testdf, type = 'class')

Lets print out the predictions:

In [ ]:
print(pred)

Lets compare these with the actuals:

In [ ]:
print(testdf$WillWait)

Make the agreement vector and calculate the accuracy which is the mean
of the agreement vector:

In [ ]:
agreement <- (pred == testdf$WillWait)
agreement
mean(agreement)